In [6]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

def process_subtasks(driver, wait, df, name):
    body = wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
    subtask_rows_count = len(body.find_elements(By.XPATH, ".//tr"))

    for i in range(subtask_rows_count):
        # ใช้ i+1 เพราะ index ของ Python เริ่มที่ 0 แต่ XPath เริ่มที่ 1
        print(f"\n--- กำลังตรวจสอบแถวที่ {i + 1} ---")
        
        try:
            # 3. สร้าง XPath สำหรับแถวปัจจุบันขึ้นมาใหม่ทุกครั้ง
            # เราต้องค้นหาจากทั้งหน้า เพราะหลังจากคลิกกลับมา DOM อาจจะเปลี่ยนไป
            current_row_xpath = f"(//table[contains(@class, 'table-zebra')]//tbody/tr)[{i + 1}]"
            
            # 4. ดึงข้อมูล %เบิกจ่าย และชื่องานจากแถวปัจจุบัน
            current_row_element = wait.until(EC.visibility_of_element_located((By.XPATH, current_row_xpath)))

            subtask_percen_dis_text = current_row_element.find_element(By.XPATH, ".//td[9]").text.strip()
            subtask_name = current_row_element.find_element(By.XPATH, ".//td[2]").text.strip()
            
            # ข้ามแถวที่ไม่มีข้อมูลเปอร์เซ็นต์
            if not subtask_percen_dis_text:
                print("  -> ไม่มีข้อมูล %เบิกจ่าย, ข้ามไป")
                continue
            
            # ค้นหาใน DataFrame
            matched = df[df['ชื่อ / ประมาณการ'] == subtask_name]
            if not matched.empty:
                project_name = matched.iloc[0]['ชื่อหน่วยงานระดับโครงการ']
                print(f"  หน่วยงาน: {project_name} | ชื่องาน: {subtask_name}... | %เบิกจ่าย: {subtask_percen_dis_text}")

                # 5. แปลงค่าและตรวจสอบเงื่อนไข
                percentage_value = float(subtask_percen_dis_text)
                if percentage_value < 100:
                    print(f"  -> [ผ่านเงื่อนไข {percentage_value}] %เบิกจ่าย < 100. กำลังคลิก 'เรียกดูข้อมูลแถบเดิม'...")
                    
                    # เงื่อนไข: ถ้า %เบิกจ่าย < 100
                    try:
                        if float(percentage_value) < 100:
                            # เข้าไปหน้าเบิกจ่าย
                            url = f"https://cenproject.rid.go.th/track/budget/451?BudgetYear=2025&BudgetSourceID=1&OrganizationName={name}&ProjectName={subtask_name}"
                            print(f"-> เข้าไปที่: {url}")
                            driver.get(url)
                            time.sleep(10)  # รอ 10 วินาที

                            # กลับมาหน้ารายการย่อย
                            back_url = f"https://cenproject.rid.go.th/track/budget/451?BudgetYear=2025&BudgetSourceID=1&OrganizationName={name}"
                            print(f"<- กลับมาที่: {back_url}")
                            driver.get(back_url)
                            wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
                        else:
                            print("ข้าม (เบิกจ่าย >= 100%)")
                    
                    except Exception as e:
                        print(f"เกิดข้อผิดพลาดขณะเข้าไปที่ URL: {e}")
                        continue
        except Exception as e:
            print(f"เกิดข้อผิดพลาดขณะประมวลผลแถวที่ {i + 1}: {e}")
            continue       


In [ ]:
process_subtasks(driver, wait)
